# Yelp Dataset preprocessing by ASAC

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install polars

In [3]:
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt

## Open Dataset

In [4]:
user_df = pl.read_ndjson("/content/drive/MyDrive/ASAC 5기/woowahan/dataset/yelp_academic_dataset_user.json")

FileNotFoundError: No such file or directory (os error 2): /content/drive/MyDrive/ASAC 5기/woowahan/dataset/yelp_academic_dataset_user.json

In [ ]:
biz_df = pl.read_ndjson("/content/drive/MyDrive/ASAC 5기/woowahan/dataset/yelp_academic_dataset_business.json")

In [ ]:
review_df = pl.read_ndjson("/content/drive/MyDrive/ASAC 5기/woowahan/dataset/yelp_academic_dataset_review.json")

In [ ]:
area_df = pl.read_csv("/content/drive/MyDrive/ASAC 5기/woowahan/dataset/active_area.csv")

In [ ]:
## polar to pandas
user_df = user_df.to_pandas()
biz_df = biz_df.to_pandas()
review_df = review_df.to_pandas()
area_df = area_df.to_pandas()

In [ ]:
biz_df.info()

##  Label Encoding id values

* review_df 테이블의 user_id, business_id, review_id를 기준으로 인코더 생성 및 인코딩
* 이후 user_df와 biz_df 테이블에 동일한 인코더 적용하여 인코딩

In [ ]:
print("유저 테이블 유저 수 : ")
print(user_df['user_id'].nunique())

print("리뷰 테이블 유저 수 : ")
print(review_df['user_id'].nunique())

print("비즈니스 테이블 비즈니스 수 : ")
print(biz_df['business_id'].nunique())

print("리뷰 테이블 비즈니스 수 : ")
print(review_df['business_id'].nunique())

In [ ]:
## 각 df의 id값 인코딩 함수

from sklearn.preprocessing import LabelEncoder

def label_encode_column(df, column, encoder=None):
    if encoder is None:
        encoder = LabelEncoder()
        df[column] = encoder.fit_transform(df[column])
    else:
        df[column] = encoder.transform(df[column])
    return encoder

In [ ]:
## review_df의 id값 인코딩 적용 후 user_id와 biz_id에도 동일한 인코더 적용

def encode_review_data(review_df, user_df, biz_df):
    # 라벨 인코더 초기화
    user_encoder = label_encode_column(review_df, 'user_id')
    biz_encoder = label_encode_column(review_df, 'business_id')
    review_encoder = label_encode_column(review_df, 'review_id')

    # 리뷰 데이터프레임에 인코더 적용
    label_encode_column(user_df, 'user_id', user_encoder)
    label_encode_column(biz_df, 'business_id', biz_encoder)

    return review_df, user_df, biz_df

In [ ]:
# 함수 호출
encoded_review_df, encoded_user_df, encoded_biz_df = encode_review_data(review_df, user_df, biz_df)

## Filtering business categories

* categories 컬럼 전처리
* main/sub라는 새로운 컬럼 생성
* main1과 main2가 'Restaurants'와 'Food'인 경우로 필터링

### categories 컬럼 전처리

In [ ]:
encoded_biz_df["categories"][141]  # 확인

In [ ]:
## 1. categories 컬럼 전처리

# split 및 공백처리
encoded_biz_df = encoded_biz_df.dropna(subset=["categories"])  # 카테고리 없는 행 제거
encoded_biz_df["categories"] = encoded_biz_df["categories"].str.split(",").apply(lambda x  :  [i.strip() for i in x] if x != None else x)

# Food에서 "Beer, Wine & Spirits"는 하나로 합쳐줘야 함
try :
  for idx, row in encoded_biz_df.iterrows() :
    for i in range(len(row["categories"])) :
      if row["categories"][i] == "Beer" :
        row["categories"][i] = "Beer, Wine & Spirits"
        del row["categories"][i+1]
        encoded_biz_df.at[idx, "categories"] = row["categories"]
        break
except : # category None인 경우
  pass

# encoded_biz_df["categories"][141]  # 확인

In [ ]:
encoded_biz_df["categories"][141]  # 확인

### main / sub 파생변수 생성

In [ ]:
## yelp_categories.json load
import json

cat_path = "/content/drive/MyDrive/ASAC 5기/woowahan/dataset/yelp_categories.json"  # 경로 변경
with open( cat_path, 'r') as file:
  yelp_cat = json.load(file)

In [ ]:
## 카테고리 딕셔너리 생성 함수
def create_category_dict(yelp_cat):
    category = {c["title"]: {"alias": c["alias"], "subs": []} for c in yelp_cat if not c["parents"]}
    for c in yelp_cat:
        if c["parents"]:
            for title, info in category.items():
                if info["alias"] == c["parents"][0] and c["title"] not in info["subs"]:
                    info["subs"].append(c["title"])
    return category


In [ ]:
## 메인 카테고리 추출 함수(최대 2개)
def extract_main_categories(business, category):
    main1, main2 = [], []
    cat_set = list(category.keys())
    for cats in business["categories"]:
        temp = [cat for cat in cats if cat in cat_set]
        main1.append(temp[0] if temp else None)
        main2.append(temp[1] if len(temp) > 1 else None)
    return main1, main2

In [ ]:
## 서브 카테고리 추출 함수
def extract_subcategories(business, category, main1, main2):
    def process_subcategories(temp):
        if len(temp) >= 2:
            return (temp[0], temp[1])
        elif temp:
            return (temp[0], None)
        else:
            return (None, None)

    main1_sub1, main1_sub2 = [], []
    main2_sub1, main2_sub2 = [], []

    for i, row in business.iterrows():
        main1_subs = category[row["main1"]]["subs"] if row["main1"] else []
        main2_subs = category[row["main2"]]["subs"] if row["main2"] else []

        temp1 = [cat for cat in row["categories"] if cat in main1_subs]
        temp2 = [cat for cat in row["categories"] if cat in main2_subs]

        main1_sub = process_subcategories(temp1)
        main2_sub = process_subcategories(temp2)

        main1_sub1.append(main1_sub[0])
        main1_sub2.append(main1_sub[1])
        main2_sub1.append(main2_sub[0])
        main2_sub2.append(main2_sub[1])

    return main1_sub1, main1_sub2, main2_sub1, main2_sub2


In [ ]:
## 함수 적용
category_dict = create_category_dict(yelp_cat)
main1, main2 = extract_main_categories(encoded_biz_df, category_dict)
encoded_biz_df["main1"], encoded_biz_df["main2"] = main1, main2

main1_sub1, main1_sub2, main2_sub1, main2_sub2 = extract_subcategories(encoded_biz_df, category_dict, main1, main2)
encoded_biz_df["main1_sub1"], encoded_biz_df["main1_sub2"], encoded_biz_df["main2_sub1"], encoded_biz_df["main2_sub2"] = main1_sub1, main1_sub2, main2_sub1, main2_sub2

encoded_biz_df.info()

### main1 / main2가 'Restaurants' or 'Food'인 비즈니스만 필터링

In [ ]:
## Restaurants, Food 필터링 함수

def filter_business_by_main_category(business, categories=["Restaurants", "Food"]):
    condition = (business["main1"].isin(categories)) | (business["main2"].isin(categories))
    return business[condition]

In [ ]:
## main1의 sub가 비었을 경우, main2가 res,food인 데이터 사용 함수

def determine_optimal_category(biz_res_fd):
    col = []
    for i, row in biz_res_fd.iterrows():
        if pd.isna(row["main1_sub1"]) and row["main2"] in ["Restaurants", "Food"] and pd.notna(row["main2_sub1"]):
            col.append([row["main2"], row["main2_sub1"]])
        elif row["main1"] not in ["Restaurants", "Food"] and row["main2"] in ["Restaurants", "Food"]:
            col.append([row["main2"], row["main2_sub1"]])
        else:
            col.append([row["main1"], row["main1_sub1"]])
    biz_res_fd["category"] = col
    return biz_res_fd

In [ ]:
## Nan값 필터링 함수

def filter_out_nan_categories(biz_res_fd):
    condition = biz_res_fd['category'].apply(lambda x: pd.isna(x[1]))
    return biz_res_fd[~condition]

In [ ]:
## 함수 적용
filtered_business = filter_business_by_main_category(encoded_biz_df)
optimized_categories = determine_optimal_category(filtered_business)
encoded_biz_df_filter_cat = filter_out_nan_categories(optimized_categories)

In [ ]:
# 결과 출력
encoded_biz_df_filtered = encoded_biz_df_filter_cat.drop(columns=['categories','main1', 'main2', 'main1_sub1','main1_sub2','main2_sub1','main2_sub2'],axis=1)

In [ ]:
encoded_biz_df_filtered

## Filtering review df

* 2017 ~ 2021 로 기간한정

* business_df의 카테고리로 필터링



### 기간 필터링

In [ ]:
encoded_review_df['date'].unique()

In [ ]:
# 날짜 형식 변경
encoded_review_df['date'] = pd.to_datetime(encoded_review_df['date'], format='%Y-%m-%d %H:%M:%S')

# 특정 연도 범위 데이터 필터링
start, end = 2017, 2021
encoded_review_df_filtered_date = encoded_review_df[(encoded_review_df['date'].dt.year >= start) & (encoded_review_df['date'].dt.year <= end)]

In [ ]:
encoded_review_df_filtered_date['date'].unique()

### 카테고리 필터링

In [ ]:
biz_list = list(encoded_biz_df_filtered["business_id"])
encoded_review_df_filtered  = encoded_review_df_filtered_date[encoded_review_df_filtered_date["business_id"].isin(biz_list)]
encoded_review_df_filtered.info()

## Create most_visite_region column

* biz_df 의 city + postal_code를 이용한 region 컬럼 생성

* review 테이블에 region 컬럼을 join 하여 user별 최빈값 count


### Create 'region' col to review table

In [ ]:
# 새로운 region 컬럼 생성 (city와 postal_code 결합)
encoded_biz_df_filtered['region'] = encoded_biz_df_filtered['city'] + ', ' + encoded_biz_df_filtered['postal_code'].astype(str)

In [ ]:
encoded_biz_df_filtered.info()

### Join review table + 'region' column

In [ ]:
# biz_df에서 business_id와 region 컬럼만 선택
biz_region_df = encoded_biz_df_filtered[['business_id', 'region']]

biz_region_df

In [ ]:
biz_region_df.info()

In [ ]:
## business_id를 기준으로 review_df_select와 biz_region_df 조인

merged_biz_rev_df = pd.merge(encoded_review_df_filtered, biz_region_df, on='business_id', how='left')

In [ ]:
# 결과 확인
merged_biz_rev_df

In [ ]:
## 최소한의 컬럼만 선택
review_join_region = merged_biz_rev_df[['review_id','user_id','business_id','region']]

In [ ]:
review_join_region

In [ ]:
import pandas as pd

# 'region' 열에서 NaN 값이 있는 행들을 필터링
nan_region_rows = review_join_region[review_join_region['region'].isna()]

# 결과 출력
print(nan_region_rows)


### Calculate most_visited_region & visit_count

In [ ]:
new_df = encoded_user_df[['user_id']]

In [ ]:
# sample_review와 sample_biz를 business_id 기준으로 병합
merged_df = pd.merge(review_join_region, new_df, on='user_id')

# user_id별로 business_id 방문 횟수 계산
visit_counts = merged_df.groupby(['user_id', 'region']).size().reset_index(name='visit_cnt')

# 가장 많이 방문한 지역을 찾기 위해 user_id별로 최대 방문 횟수 필터링
max_visits = visit_counts.loc[visit_counts.groupby('user_id')['visit_cnt'].idxmax()]

# 만약 최빈값이 동일한 경우, 날짜를 기준으로 최신 방문 지역을 찾기 위해 user_id별 최신 리뷰 필터링
latest_reviews = merged_df.loc[merged_df.groupby(['user_id', 'region'])['date'].idxmax()]

# 최대 방문 지역과 최신 리뷰 데이터를 병합
final_df = pd.merge(max_visits, latest_reviews[['user_id', 'region', 'date']], on=['user_id', 'region'], how='left')

# user_df에 'most_visited_region'과 'visit_cnt' 컬럼 추가
new_df = pd.merge(new_df, final_df[['user_id', 'region', 'visit_cnt']], on='user_id', how='left')
new_df = new_df.rename(columns={'region': 'most_visited_region'})


In [ ]:
# user_id를 기준으로 most_visited_region과 visit_cnt를 merged_df에 병합
encoded_user_df_local = pd.merge(encoded_user_df, new_df[['user_id', 'most_visited_region', 'visit_cnt']], on='user_id', how='left')

encoded_user_df_local

In [ ]:
# ## 최빈값을 모두 리스트로 반환

# def get_all_modes(x):
#     value_cnt = x.value_counts()    # 빈도수 계산
#     max_freq = value_cnt.max()      # 최빈값 찾기
#     modes = list(value_cnt[value_cnt == max_freq].index)    # 최빈값 리스트 변환
#     return modes, max_freq          # 최빈값 리스트, 빈도수 반환

# def calculate_active_area_to_list(df):
#     # groupby를 이용해 region 그룹화
#     results = df.groupby('user_id')['region'].apply(lambda x: get_all_modes(x)).reset_index()
#     # 각 그룹에 대해 get_all_modes함수 적용
#     results[['most_visited_regions', 'visit_count']] = pd.DataFrame(results['region'].tolist(), index=results.index)
#     results.drop(columns='region', inplace=True)
#     results.columns = ['user_id', 'most_visited_regions', 'visit_count']
#     return results


In [ ]:
# # 함수 호출
# active_area_df = calculate_active_area_to_list(review_join_region)


In [ ]:
# ## 최빈값 한개 추출

# ## group by를 통한 최빈값 계산
# def calculate_active_area(df):
#     # user_id별 region의 최빈값과 그 횟수 계산
#     most_visited = df.groupby('user_id')['region'].agg(lambda x: x.value_counts().idxmax()).reset_index()
#     visit_counts = df.groupby('user_id')['region'].agg(lambda x: x.value_counts().max()).reset_index()

#     # 결과를 병합하여 최종 데이터프레임 생성
#     active_area_df = pd.merge(most_visited, visit_counts, on='user_id')
#     active_area_df.columns = ['user_id', 'most_visited_region', 'visit_count']

#     return active_area_df

In [ ]:
# # 함수 호출
# active_area_df = calculate_active_area(review_join_region)


In [ ]:
# active_area_df.info()

In [ ]:
# active_area_df['visit_count'].value_counts()

In [ ]:
print(encoded_user_df_local['visit_cnt'].describe())


In [ ]:
import matplotlib.pyplot as plt

plt.hist(encoded_user_df_local['visit_cnt'], bins=30, color='skyblue', edgecolor='k')
plt.title('Histogram of Visit Count')
plt.xlabel('Visit Count')
plt.ylabel('Frequency')
plt.show()


In [ ]:
encoded_user_df_local['most_visited_region'].iloc[352]


In [ ]:
# DF 명
# encoded_user_df
# encoded_biz_df_filtered
# encoded_review_df_filtered
# active_area_df

## Drop columns & Add new columns

* user_id 별 review_count_user, average_stars_user add

* biz_id 별 review_count_biz, average_stars_biz add

* user_df의 review_count, avg_stars drop

* biz_df의 review_count, hours drop

* review_df text_length add

In [ ]:
## 함수 1 - 그룹별 리뷰 갯수 반환 함수
def add_review_count(df, encoded_review_df_filtered, group_col, count_col_name):
    # 그룹별 리뷰 개수 계산
    review_counts = encoded_review_df_filtered.groupby(group_col).size().reset_index(name=count_col_name)

    # 원본 DataFrame에 병합
    df = pd.merge(df, review_counts, how='left', on=group_col)

    # 결측값의 개수 계산
    missing_count = df[count_col_name].isna().sum()

    # 결측값의 개수를 출력
    print(f"리뷰가 없는 {group_col} 수: {missing_count}")

    # 결측값을 0으로 채움 (리뷰가 없는 경우)
    df[count_col_name].fillna(0, inplace=True)

    return df

In [ ]:
## 함수 2 - 그룹별 평균 별점 계산 함수
def calculate_avg_rating(df, group_col, rating_col, new_col_name):
    # 그룹별 평균 별점 계산
    avg_ratings = df.groupby(group_col)[rating_col].mean().reset_index(name=new_col_name)
    return avg_ratings

In [ ]:
## 1. user_id 별 리뷰 수와 별점 평균 add

# 리뷰 수 추가
encoded_user_df_local = add_review_count(encoded_user_df_local, encoded_review_df_filtered, 'user_id', 'review_count_user')

# 평균 별점 계산
avg_user_ratings = calculate_avg_rating(encoded_review_df_filtered, 'user_id', 'stars', 'average_stars_user')

# 평균 별점을 원본 DataFrame에 병합
encoded_user_df_local = pd.merge(encoded_user_df_local, avg_user_ratings, how='left', on='user_id')



In [ ]:
encoded_user_df_local.info()

In [ ]:
## 2. biz_id 별 리뷰 수와 별점 평균 add

# 리뷰 수 추가
encoded_biz_df_filtered = add_review_count(encoded_biz_df_filtered, encoded_review_df_filtered, 'business_id', 'review_count_biz')

# 평균 별점 계산
avg_biz_ratings = calculate_avg_rating(encoded_review_df_filtered, 'business_id', 'stars', 'average_stars_biz')

# 평균 별점을 원본 DataFrame에 병합
encoded_biz_df_filtered = pd.merge(encoded_biz_df_filtered, avg_biz_ratings, how='left', on='business_id')



In [ ]:
encoded_biz_df_filtered

In [ ]:
## 3. 불필요한 컬럼 drop
encoded_user_df = encoded_user_df.drop(columns=['review_count','average_stars'], axis=1)
encoded_biz_df_filtered = encoded_biz_df_filtered.drop(columns=['review_count', 'stars','hours'],axis=1)

In [ ]:
## 4. review_df text_length 추가

import nltk
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')

# \n, (?), (?, ?)을 공백으로 대체하는 코드
encoded_review_df_filtered['text'] = encoded_review_df_filtered['text'].str.replace(r'\n', ' ', regex=True)
encoded_review_df_filtered['text'] = encoded_review_df_filtered['text'].str.replace(r'\(\?\)', ' ', regex=True)
encoded_review_df_filtered['text'] = encoded_review_df_filtered['text'].str.replace(r'\(\?, \?\)', ' ', regex=True)


# 문장으로 분리
tokenizer = PunktSentenceTokenizer()


# 문장 개수 계산 및 데이터프레임에 추가
encoded_review_df_filtered['text_length'] = encoded_review_df_filtered['text'].apply(lambda x: len(tokenizer.tokenize(x)))


## Check Final DataFrame

In [ ]:
encoded_user_df_local.info()

In [ ]:
encoded_user_df_local.head()

In [ ]:
encoded_biz_df_filtered.info()

In [ ]:
encoded_biz_df_filtered.head()

In [ ]:
encoded_review_df_filtered.info()

In [ ]:
encoded_review_df_filtered.head()

In [ ]:
active_area_df.info()

In [ ]:
active_area_df.head()

## Save Final DataFrame

In [ ]:
# # DataFrame 저장
# encoded_user_df_local.to_csv('/content/drive/MyDrive/ASAC 5기/woowahan/dataset/final/encoded_user_df_local.csv', index=False)
# encoded_biz_df_filtered.to_csv('/content/drive/MyDrive/ASAC 5기/woowahan/dataset/final/encoded_biz_df_filtered.csv', index=False)
# encoded_review_df_filtered.to_csv('/content/drive/MyDrive/ASAC 5기/woowahan/dataset/final/encoded_review_df_filtered.csv', index=False)
# active_area_df.to_csv('/content/drive/MyDrive/ASAC 5기/woowahan/dataset/final/active_area_df.csv', index=False)